In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline 

In [ ]:
from yaml import load as yaml_load
import os
import sys
import json
sys.path.append('../')
import findspark
findspark.init()
import pyspark
from pyspark.sql.types import LongType
from pyspark.sql import SparkSession
import pyspark.sql.functions as func
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression, GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import StandardScaler, VectorAssembler, VectorIndexer, StandardScaler
from pyspark.sql.functions import *
from pyspark.sql.functions import pandas_udf, PandasUDFType
import pandas as pd
from pyspark.sql.functions import col, pandas_udf, udf
from pyspark.sql.types import LongType, StringType, FloatType, IntegerType, DoubleType
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
import re
spark = SparkSession.builder.master("local").appName("Data cleaning").getOrCreate()

In [ ]:
from src.data.make_dataset import LoadDataframe

In [ ]:
def _load_config_file(config_file):
    """
    Load configuration file
    :param config_file: is the configuration file
    :return: configuration
    :rtype: dict
    """
    with open(config_file) as yml_config:
        return yaml_load(yml_config)

def _build_configuration(config_file):
    """
    Build the operation configuration dict
    :param config_file: is the path to the yaml config_file
    :type: string
    :return: config: global configuration
    :rtype dict
    """
    # yaml config
    config = _load_config_file(config_file)
    return config
def visualisation_prediction(y_test, y_pred):
    import matplotlib
    import matplotlib.pyplot as plt
    matplotlib.rc('xtick', labelsize=30) 
    matplotlib.rc('ytick', labelsize=30) 
    fig, ax = plt.subplots(figsize=(50, 40))
    plt.style.use('ggplot')
    plt.plot(y_pred, y_test, 'ro')
    plt.xlabel('Predicted Crime', fontsize = 30)
    plt.ylabel('Actual Crime', fontsize = 30)
    plt.title('Predicted Y (Crimes) to the Actual Y (Crimes)', fontsize = 30)
    ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=4)
    
    
def duration_day_func(x):
    """
    :return:
    """
    from astral import Astral
    city_name = 'Chicago'
    a = Astral()
    a.solar_depression = 'civil'
    city = a[city_name]
    sun = city.sun(date=x, local=True)
    return float((sun['sunset'] - sun['sunrise']).total_seconds())

extract_blok = udf(lambda x : re.findall(r"(\w+)$", x)[0], StringType())
isStreet = udf(lambda x :  1 if x in ['ST', 'St', 'st'] else 0, DoubleType())
isAV = udf(lambda x : 1 if x in ['Ave', 'AV', 'AVE'] else 0, DoubleType())
isBLVD = udf(lambda x : 1 if x in ['BLVD'] else 0, DoubleType())
isRD = udf(lambda x : 1 if x in ['RD'] else 0, DoubleType())
isPL = udf(lambda x : 1 if x in ['PL', 'pl'] else 0, DoubleType())
isBROADWAY = udf(lambda x : 1 if x in ['BROADWAY', 'Broadway'] else 0, DoubleType())
isPKWY = udf(lambda x : 1 if x in ['PKWY', 'Pkwy'] else 0, DoubleType())
duration_day_udf = udf(lambda x :   duration_day_func(x),  DoubleType())

config_file = "/home/ml/Documents/crimes_chigaco/config/config.yml"
config = _build_configuration(config_file)

In [ ]:
%%time
obj_df_loaded = LoadDataframe(config, '2013', '2014')

In [ ]:
df_temp = obj_df_loaded.df_temperature()
df_sky  = obj_df_loaded.df_sky()
df_socio = obj_df_loaded.df_socio()

In [ ]:
features_socio = ['pct_housing_crowded',
 'pct_households_below_poverty','pct_age16_unemployed',
 'pct_age25_no_highschool',
 'pct_not_working_age',
 'per_capita_income',
 'hardship_index']
for f in features_socio:
    df_socio = df_socio.withColumn(f, df_socio[f].cast(FloatType()))

In [ ]:
df_crime = obj_df_loaded.df_crime()
df_crime_socio = df_crime.join(df_socio, ['community_area_number'], "inner")

In [ ]:
df_crime_socio = df_crime_socio.withColumn("block_extract", extract_blok(df_crime_socio.block))

In [ ]:
df_crime_socio = df_crime_socio.withColumn("isStreet", isStreet(df_crime_socio.block_extract)).withColumn("isAV", isAV(df_crime_socio.block_extract)).withColumn("isBLVD", isBLVD(df_crime_socio.block_extract)).withColumn("isRD", isRD(df_crime_socio.block_extract)).withColumn("isPL", isPL(df_crime_socio.block_extract)).withColumn("isBROADWAY", isBROADWAY(df_crime_socio.block_extract)).withColumn("isPKWY", isPKWY(df_crime_socio.block_extract))

In [ ]:
df_crime_socio = df_crime_socio.withColumn('duree_day', duration_day_udf('date'))

In [ ]:
df_crime_socio = (
df_crime_socio.withColumn("month", func.month(func.col("date"))).
withColumn("year", func.year(func.col("date"))).
withColumn("day", func.dayofmonth(func.col("date"))).
withColumn("hour", func.hour(func.col("date"))).withColumn("minute", func.minute(func.col("date"))).
withColumn("dayofmonth", func.dayofmonth(func.col("date"))).   
withColumn("dayofyear", func.dayofyear(func.col("date"))).
withColumn("dayofweek", func.dayofweek(func.col("date")))
)

In [ ]:
df_total = df_crime_socio.join(df_temp, ['year', 'month','day','hour'], how = "left")

In [ ]:
df_total = df_total.withColumn('latitude', df_total['latitude'].cast(FloatType()))
df_total = df_total.withColumn('longitude', df_total['longitude'].cast(FloatType()))

In [ ]:
list_of_crimes = config["List_of_crimes_prediction"]["with_merge_pred"]

In [ ]:
df = df_total.where(col('primary_type').isin(list_of_crimes))

In [ ]:
df.printSchema()

In [ ]:
categoricalColumns = ['domestic']
numericCols = ['year', 'month', 'day', 'hour','minute', 'latitude', 'longitude', 'isStreet', 'isAV', 'isBLVD', 'isRD', 'isPL', 'isBROADWAY', 'isPKWY']
stages = []

In [ ]:
for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
    encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]

In [ ]:
label_stringIdx = StringIndexer(inputCol = 'primary_type', outputCol = 'label')
stages += [label_stringIdx]

In [ ]:
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols

In [ ]:
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")

In [ ]:
stages += [assembler]

In [ ]:
stages

In [ ]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages = stages)

In [ ]:
pipelineModel = pipeline.fit(df)

In [ ]:
df = pipelineModel.transform(df)

In [ ]:
selectedCols = ['label', 'features'] + categoricalColumns + numericCols

In [ ]:
df = df.select(selectedCols)
df.printSchema()

In [ ]:
df.show(5)

In [ ]:
df.groupBy('label').count().orderBy('count').show()

In [ ]:
df.show(5)